In [ ]:
import io
from init_notebook import *
from src.clipig.app.images import LImage
from src.util.files.filestream import Filestream
from src.algo.sdf.two_d.util import perlin_noise_2d

In [ ]:
with Filestream("../src/clipig/projects/wang-edge-2x2.clipig.tar") as fs:
    limage = LImage()
    limage.load_from_filestream(fs, "task_00/limage/config.yaml")
tiling = limage.tiling
template = limage.layers[1].to_torch()
with Filestream("../src/clipig/projects/wang-ec2-grass-water.clipig.tar") as fs:
    limage = LImage()
    limage.load_from_filestream(fs, "task_00/limage/config.yaml")
tiling_ec = limage.tiling
template_ec = limage.layers[0].to_torch()
#limage.layers[0].to_pil()

In [ ]:
VF.to_pil_image(tiling.render_tile_map(
    template=template,
    map=tiling.create_map_stochastic_scanline((20, 20)),
))

In [ ]:
def num_bits(x: int):
    num = 0
    while x:
        if x & 1:
            num += 1
        x = x >> 1
    return num


def create_map_stochastic(
        self,
        size: Tuple[int, int],
        seed: Optional[int] = None,
):
    rng = random.Random(seed)
    tiles_outside = []
    tiles_inside = []
    max_elements = max(a.num_elements() for a in self.attributes_map.values())
    print("X", max_elements)
    for idx, a in self.attributes_map.items():
        n = a.num_elements()
        if n == 0:
            tiles_outside.append(idx)
        if n == max_elements:
            tiles_inside.append(idx)
    noise = perlin_noise_2d(
        shape=(size[1], size[0]),
        res=(size[1]//3, size[0]//3),
    )    
    map = np.tile(np.array([[None]], dtype=np.object_), noise.shape)
    for y, row in enumerate(map):
        for x, v in enumerate(row):
            if noise[y, x] > .1:
                map[y, x] = repr(rng.choice(tiles_inside))
            if noise[y, x] < -.3:
                map[y, x] = repr(rng.choice(tiles_outside))
    #map[noise > .1] = repr(tile_inside)
    #map[noise < -.3] = repr(tile_outside)
    all_tiles = list(self.attributes_map.keys())
    for i in range(size[0]*size[1] // 2):
        x, y = rng.randrange(size[0]), rng.randrange(size[1])
        if map[y, x] is None or rng.uniform(0, 1) < .1:
            map[y, x] = repr(rng.choice(all_tiles))
    map = [
        [eval(x) if x else None for x in row]
        for row in map
    ]
    self.set_map_default(map, rng.choice(tiles_outside))
    for i in range(109):
        has_errors, is_solved = self.resolve_map(map)
        if is_solved:
            break
    print(i, has_errors, is_solved)
    return map

if 0:
    display(VF.to_pil_image(tiling.render_tile_map(
        template=template,
        map=create_map_stochastic(tiling, (8, 8)),
    )))
else:
    display(VF.to_pil_image(resize(tiling_ec.render_tile_map(
        template=template_ec,
        map=create_map_stochastic(tiling_ec, (18, 18)),
    ), 2)))

In [ ]:
VF.to_pil_image(resize(tiling_ec.render_tile_map(
    template=template_ec,
    map=create_map_stochastic(tiling_ec, (18, 18)),
), 2))

In [ ]:
def num_bits(x: int):
    num = 0
    while x:
        if x & 1:
            num += 1
        x = x >> 1
    return num

def create_map_stochastic(
        self,
        size: Tuple[int, int],
        seed: Optional[int] = None,
):
    rng = random.Random(seed)

    possible_tiles = list(self.attributes_map.keys())
    probabilities = []
    for a in self.attributes_map.values():
        n = a.num_elements()
        p = 1
        if n == 0 or n == 4:
            p = 50
        probabilities.append(p)
    print(probabilities)
    print(rng.choices(possible_tiles, weights=probabilities, k=30))
    map = [[None] * size[0] for _ in range(size[1])]

    positions = [(x, y) for y in range(size[1]) for x in range(size[0])]
    rng.shuffle(positions)
    for x, y in positions:
        for try_idx in range(len(possible_tiles) * 2):
            tile_idx = rng.choices(possible_tiles, weights=probabilities)[0]
                    
            attrs = self.tile_attributes(tile_idx)

            if y > 0 and map[y-1][x] is not None and not attrs.matches_top(self.tile_attributes(map[y - 1][x])):
                continue
            if y < size[1]-1 and map[y+1][x] is not None and not attrs.matches_bottom(self.tile_attributes(map[y + 1][x])):
                continue
            if x > 0 and map[y][x-1] is not None and not attrs.matches_left(self.tile_attributes(map[y][x - 1])):
                continue
            if x < size[0]-1 and map[y][x+1] is not None and not attrs.matches_right(self.tile_attributes(map[y][x + 1])):
                continue

            map[y][x] = tile_idx
            break
    
    for row in map:
        for i, v in enumerate(row):
            if v is None:
                row[i] = (0, 0)

    return map
    
VF.to_pil_image(tiling.render_tile_map(
    template=template,
    map=create_map_stochastic(tiling, (20, 20)),
))